# Amplitudes analysis

Importing all necessary stuff:

In [2]:
import numpy as np
from statistics import stdev
from matplotlib import pyplot as plt #this is the traditional method

We now collect the data from the files:

In [12]:
file_folder = 'data/actual data/Frequency response/'
file_name = 'amplitudes1.csv'
data_file = file_folder + file_name

frequencies, amplitudeGen, amplitudeR, amplitudeL, amplitudeC  = np.loadtxt(data_file, delimiter = ',', skiprows = 1, usecols = (0, 1, 2, 3, 4), unpack = True)

# amplitudes values are in mV units
amplitudeGen *= 10**(-3)
amplitudeR *= 10**(-3)
amplitudeL *= 10**(-3)
amplitudeC *= 10**(-3)

We now define the function we are going to fit the data with: $$ V_R = \frac{R_rV_0}{\sqrt{R^2+{ \left(\omega L - \frac{1}{\omega C}\right)}^2}}$$
$$ V_L = \frac{\omega L V_0}{\sqrt{R^2+{ \left(\omega L - \frac{1}{\omega C}\right)}^2}}$$
$$ V_C = \frac{\frac{V_0}{\omega C}}{\sqrt{R^2+{ \left(\omega L - \frac{1}{\omega C}\right)}^2}}$$

 $\omega = 2 \pi f$ with $f$ being the sinusoidal generator frequency. We've chosen to estimate the uncertainty associated to the amplitude measures with the sample standard deviation $\sigma$ since how the DAQ program outputs the amplitude values is not known.


In [ ]:
def R_amp_model(f, R_r , R, L, C, V_0):
    num = R_r * V_0
    den = np.sqrt( R**2 + (2*np.pi*f * L - 1/(2*np.pi*f * C))**2)
    return num / den
def L_amp_model(f, R, L, C, V_0):
    num = 2*np.pi*f * L * V_0
    den = np.sqrt( R**2 + (2*np.pi*f * L - 1/(2*np.pi*f * C))**2)
    return num / den
def C_amp_model(f, R, L, C, V_0):
    num = V_0 / (2*np.pi*f * C)
    den = np.sqrt( R**2 + (2*np.pi*f * L - 1/(2*np.pi*f * C))**2)
    return num / den

We now have to make our initial guess of the fitting parameters

In [ ]:
#initial guesses(theoretical values)
R_r = 996.7
R = 1.17252e+03
L = 4.741e-02
C = 1.46e-09
V_0 = 0.5
initial_guess_R = np.array([R_r , R, L, C, V_0])
initial_guess_L = np.array([R, L, C, V_0])
initial_guess_C = np.array([R, L, C, V_0])
# initial_error = np.array([delta_V0, delta_theta, delta_V1])
old_fit_R = np.copy(initial_guess_R)  # save a copy to compare new with old
old_fit_L = np.copy(initial_guess_L)  # save a copy to compare new with old
old_fit_C = np.copy(initial_guess_C)  # save a copy to compare new with old

# estimated_error = photodiode_error(theta, delta_V0, delta_theta, delta_V1,
#                                    V0, theta0)    #propagate uncertainty using initial values

We now compute $\sigma$

In [ ]:
from statistics import stdev

amplitude_stdev = stdev(amplitudeGen)


amplitude_stdev

### Curve fit routine
We are now going to use curve_fit function from scipy

In [ ]:
# load curve_fit routine from scipy
from scipy.optimize import curve_fit # import method used here

# the method returns two arrays: one with the resulting fit parameters and then the covariance matrix
fit_R, covariance_R = curve_fit(R_amp_model, frequencies_gen, amplitude_R, p0 = initial_guess_R,sigma=amplitude_stdev,absolute_sigma=True)
fit_L, covariance_L = curve_fit(L_amp_model, frequencies_gen, amplitude_L, p0 = initial_guess_L,sigma=amplitude_stdev,absolute_sigma=True)
fit_C, covariance_C = curve_fit(C_amp_model, frequencies_gen, amplitude_C, p0 = initial_guess_C,sigma=amplitude_stdev,absolute_sigma=True)

#the elements on the the diagonal represent the errors on the fit parameters
error_R = np.sqrt(np.diag(covariance_R))
error_L = np.sqrt(np.diag(covariance_L))
error_C = np.sqrt(np.diag(covariance_C))

print('R\n')
print(old_fit_R)
print(fit_R)
print('L\n')
print(old_fit_L)
print(fit_L)
print('C\n')
print(old_fit_C)
print(fit_C)

#setting the current fit results as old
old_fit_R = np.copy(fit_R)
old_fit_L = np.copy(fit_L)
old_fit_C = np.copy(fit_C)

Now printing out the first fit results

In [ ]:
print()
print('R_r    = ','{:.3f}'.format(fit_R[0]), '±', '{:.5f}'.format(error_R[0]), ' $\Omega$')
print('R    = ','{:.3f}'.format(fit_R[1]), '±', '{:.6f}'.format(error_R[1]), ' $\Omega$')
print('L    = ','{:.5f}'.format(fit_R[2]), '±', '{:.3f}'.format(error_R[2]), ' Henry')
print('C    = ','{:.12f}'.format(fit_R[3]), '±', '{:.12f}'.format(error_R[3]), ' F')
print('V_0    = ','{:.3f}'.format(fit_R[4]), '±', '{:.3f}'.format(error_R[4]), ' V')

print()
print('R    = ','{:.3f}'.format(fit_L[0]), '±', '{:.3f}'.format(error_L[0]), ' $\Omega$')
print('L    = ','{:.5f}'.format(fit_L[1]), '±', '{:.3f}'.format(error_L[1]), ' Henry')
print('C    = ','{:.12f}'.format(fit_L[2]), '±', '{:.12f}'.format(error_L[2]), ' F')
print('V_0    = ','{:.3f}'.format(fit_L[3]), '±', '{:.3f}'.format(error_L[3]), ' V')

print()
print('R    = ','{:.3f}'.format(fit_C[0]), '±', '{:.3f}'.format(error_C[0]), ' $\Omega$')
print('L    = ','{:.5f}'.format(fit_C[1]), '±', '{:.3f}'.format(error_C[1]), ' Henry')
print('C    = ','{:.12f}'.format(fit_C[2]), '±', '{:.12f}'.format(error_C[2]), ' F')
print('V_0    = ','{:.3f}'.format(fit_C[3]), '±', '{:.3f}'.format(error_C[3]), ' V')

We now iterate the process to find new fitting parameters until the values(amplitudes) become stable

In [ ]:
new_initial_values_R = np.array([fit_R[0], fit_R[1], fit_R[2],fit_R[3],fit_R[4]])
new_initial_values_L = np.array([fit_L[0], fit_L[1], fit_L[2],fit_L[3]])
new_initial_values_C = np.array([fit_C[0], fit_C[1], fit_C[2],fit_C[3]])

# estimated_error = photodiode_error(theta, delta_V0, delta_theta, delta_V1,
#                                    fit_R[0], fit_R[1]) # propagate error using new values for V0, etc

fit_R, covariance_R = curve_fit(R_amp_model, frequencies_gen, amplitude_R, p0 = new_initial_values_R,sigma=amplitude_stdev,absolute_sigma=True)
fit_L, covariance_L = curve_fit(L_amp_model, frequencies_gen, amplitude_R, p0 = new_initial_values_L,sigma=amplitude_stdev,absolute_sigma=True)
fit_C, covariance_C = curve_fit(C_amp_model, frequencies_gen, amplitude_R, p0 = new_initial_values_C,sigma=amplitude_stdev,absolute_sigma=True)

error_R = np.sqrt(np.diag(covariance_R))
error_L = np.sqrt(np.diag(covariance_L))
error_C = np.sqrt(np.diag(covariance_C))

print('R\n')
print(old_fit_R)
print(fit_R)
print('L\n')
print(old_fit_L)
print(fit_L)
print('C\n')
print(old_fit_C)
print(fit_C)

old_fit_R = np.copy(fit_R)
old_fit_L = np.copy(fit_L)
old_fit_C = np.copy(fit_C)


### Fit accuracy
Ok now that we've found the best-fit parameters we are interested in the

In [ ]:

from scipy.stats import chi2

V_R_model = R_amp_model(frequencies_gen, fit_R[0], fit_R[1], fit_R[2], fit_R[3],fit_R[4])
residual_R = amplitude_R - V_R_model
V_L_model = L_amp_model(frequencies_gen, fit_L[0], fit_L[1], fit_L[2], fit_L[3])
residual_L = amplitude_L - V_L_model
V_C_model = C_amp_model(frequencies_gen, fit_C[0], fit_C[1], fit_C[2], fit_C[3])
residual_C = amplitude_C - V_C_model

# data_uncertainty = photodiode_error(theta, delta_V0, delta_theta, delta_V1, fit_R[0], fit_R[1])

# in our case the uncertainty is estimated to be sigma
data_uncertainty = amplitude_stdev

chisq_R = sum((residual_R/ data_uncertainty)**2) #typo corrected
degrees_of_freedom_R = len(residual_R) - len(initial_guess_R)
reduced_chisq_R = chisq_R / degrees_of_freedom_R  # this should be close to one
CDF_R = chi2.cdf(chisq_R, degrees_of_freedom_R)   # this should be close to 50 percent

chisq_L = sum((residual_L/ data_uncertainty)**2) #typo corrected
degrees_of_freedom_L = len(residual_L) - len(initial_guess_L)
reduced_chisq_L = chisq_L / degrees_of_freedom_L  # this should be close to one
CDF_L = chi2.cdf(chisq_L, degrees_of_freedom_L)   # this should be close to 50 percent

chisq_C = sum((residual_C/ data_uncertainty)**2) #typo corrected
degrees_of_freedom_C = len(residual_C) - len(initial_guess_C)
reduced_chisq_C = chisq_C / degrees_of_freedom_C  # this should be close to one
CDF_C = chi2.cdf(chisq_C, degrees_of_freedom_C)   # this should be close to 50 percent

print('chi-square         = ',chisq_R)
print('degrees of freedom = ',degrees_of_freedom_R)
print('reduced chi-square = ',reduced_chisq_R)
print('fractional probability of chisq ≤', chisq_R, 'for ', degrees_of_freedom_R, 'dof is', CDF_R)

print('chi-square         = ',chisq_L)
print('degrees of freedom = ',degrees_of_freedom_L)
print('reduced chi-square = ',reduced_chisq_L)
print('fractional probability of chisq ≤', chisq_L, 'for ', degrees_of_freedom_L, 'dof is', CDF_L)

print('chi-square         = ',chisq_C)
print('degrees of freedom = ',degrees_of_freedom_C)
print('reduced chi-square = ',reduced_chisq_C)
print('fractional probability of chisq ≤', chisq_C, 'for ', degrees_of_freedom_C, 'dof is', CDF_C)

Now importing [matplotlib](https://matplotlib.org) used for graphical output

In [ ]:
%matplotlib inline

plt.figure(figsize = (11,8))                  #specify figure size as 7 x 5 inches
#for default size, type plt.figure()

plt.xlabel(r"$f$ [Hz]", fontsize = 18) #label axis (using LaTeX commands)
plt.ylabel(r"$V_R$ [volts]", fontsize = 18)   #use 18 point font for label text

# plot the data as before in blue
plt.errorbar(frequencies_gen, amplitude_R,
             xerr=None, yerr=amplitude_stdev,
             linestyle = 'none',
             color = 'blue',
             capsize = 3, capthick = 1, label = "data")

#create curve showing fit to data
freq_fit = np.linspace(5000, 35000, 50)
VR_fit = R_amp_model(freq_fit, fit_R[0], fit_R[1], fit_R[2],fit_R[3],fit_R[4])

#plot the curve fit in red
plt.errorbar(freq_fit, VR_fit, xerr = None, yerr = amplitude_stdev, color = 'red', label = 'fit' )
plt.xlim(4900 , 36000)
plt.ylim(0.0,0.25)
# plt.xticks([0, 30, 60, 90, 120, 150, 180, 210, 240, 270, 300, 330, 360],
#            ('0', '', '', 90,  '',  '', 180,  '',  '', 270,  '',  '', 360))
plt.legend(loc = 'best')

plt.savefig('Resistor-amplitude.pdf')
plt.show()
